# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [44]:
# #langchain library
# !pip3 install langchain==0.1.0
# #LLM
# !pip3 install accelerate==0.25.0
# !pip3 install transformers==4.36.2
# !pip3 install bitsandbytes==0.41.2
# #Text Embedding
# !pip3 install sentence-transformers==2.2.2
# !pip3 install InstructorEmbedding==1.0.1
# #vectorstore
# !pip3 install pymupdf==1.23.8
# !pip3 install faiss-gpu==1.7.2
# !pip3 install faiss-cpu==1.7.4

In [45]:
# !python3 -m pip install torch==2.0.1 transformers==4.35.0 langchain==0.1.11
# !pip3 install protobuf==3.20.0

In [46]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [47]:
# from langchain import PromptTemplate

# prompt_template = """
#     I'm your clone myself chatbot. You can ask anything according to yourself. Sometimes, it is ok to feel loss, but remember to speak to 
#     who might knock in a sane mind again. You can ask whether who you are ? what are your studies ? what are your interest ? what expertis areas ? 
#     No matter you feel uncomfortble, please reach out to me, ok ? 
#     {context}
#     Question: {question}
#     Answer:
#     """.strip()

# PROMPT = PromptTemplate.from_template(
#     template = prompt_template
# )

# PROMPT
# #using str.format 
# #The placeholder is defined using curly brackets: {} {}



In [48]:
# PROMPT.format(
#     context = "Vorameth is interest in many areas for exmaple machine learning, cybersecurity, or even network programming.",
#     question = "What are the areas that Vorameth interest in ? "
# )

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [49]:
from langchain.document_loaders import PyMuPDFLoader,WebBaseLoader

# nlp_docs = '../docs/pdf/SpeechandLanguageProcessing_3rd_07jan2023.pdf'

nlp_docs = "Vorameth'sCV.pdf"

web_loader_1 = WebBaseLoader("https://www.researchgate.net/profile/Vorameth-Reantongcome")
webs1 = web_loader_1.load()


web_loader_2 = WebBaseLoader("https://www.linkedin.com/in/vorameth-reantongcome/")
webs2 = web_loader_2.load()


pdf_loader = PyMuPDFLoader(nlp_docs)
pdfs = pdf_loader.load()

documents = pdfs + webs1 + webs2

In [50]:
# documents

In [51]:
len(documents)

4

In [52]:
documents[1]

Document(metadata={'source': "Vorameth'sCV.pdf", 'file_path': "Vorameth'sCV.pdf", 'page': 1, 'total_pages': 2, 'format': 'PDF 1.3', 'title': "Vorameth'sCV", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Pages', 'producer': 'macOS Version 15.3.1 (Build 24D70) Quartz PDFContext', 'creationDate': "D:20250310171528Z00'00'", 'modDate': "D:20250310171528Z00'00'", 'trapped': ''}, page_content='Assessed the website applications whether they contain any vulnerabilities which may lead to initial \nattack process  \n• Others \nBeing a representative joining TB-CERT CyberCombat Event which was the competition about security \nareas (CTF) \nJune — August 2019 \n‣ Internship, Nara Institute of Science and Technology; Nara, Japan \n• Implementing Blockchain concept to secure multi-tenants cloud computing \n• Using Trufﬂe framework to integrating the smart contract for ﬂowing the transaction on cloud \n• Proving the concept with simulated tenants and virtual machines as the smart contract \

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [53]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [54]:
doc[1]

Document(metadata={'source': "Vorameth'sCV.pdf", 'file_path': "Vorameth'sCV.pdf", 'page': 0, 'total_pages': 2, 'format': 'PDF 1.3', 'title': "Vorameth'sCV", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Pages', 'producer': 'macOS Version 15.3.1 (Build 24D70) Quartz PDFContext', 'creationDate': "D:20250310171528Z00'00'", 'modDate': "D:20250310171528Z00'00'", 'trapped': ''}, page_content='• Project on Machine Learning  \n• CO2 Emission Prediction - The project apply datasets from government open data website for \npredicting the trend of CO2 emission from each natural resource. This project could help government \nagency to predict the trend of CO2 based on controlled resource which could assist on policy \nestablishment. \n• Current Studying in Business Intelligence Analytics, Database Modeling and  Management, Data \nEngineering and Machine Learning Operation, and Natural Language Understanding \nJuly 2020 - July 2024 \n‣ Employee, Bank of Thailand; Bangkok, Thailand \n• Thr

In [55]:
len(doc)

10

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [ ]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

# model_name = 'hkunlp/instructor-base'

# model_name = 'hkunlp/instructor-large'

model_name = 'hkunlp/instructor-xl' 

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

load INSTRUCTOR_Transformer
max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [57]:
#locate vectorstore
vector_path = 'vector-store/'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

vector_path_1 = 'vector-store-lance/'
if not os.path.exists(vector_path_1):
    os.makedirs(vector_path_1)
    print('create path done')

vector_path_2 = 'vector-store-chroma/'
if not os.path.exists(vector_path_2):
    os.makedirs(vector_path_2)
    print('create path done')

In [58]:
#save vector locally
from langchain.vectorstores import FAISS,LanceDB
from langchain_chroma import Chroma
import lancedb

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'myself_db'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'myself' #default index
)
db = lancedb.connect(os.path.join(vector_path_1,db_file_name)) 
lancedb_vectordb = LanceDB.from_documents(
    documents=doc,
    embedding=embedding_model,
    connection=db
)

chroma_vectordb = Chroma.from_documents(
    documents=doc,
    embedding=embedding_model,
    persist_directory=os.path.join(vector_path_2,db_file_name)  # Auto-saves
)


### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [59]:
#calling vector from local
vector_path = 'vector-store/'
db_file_name = 'myself_db'

from langchain.vectorstores import FAISS,LanceDB


vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'myself', #default index
     allow_dangerous_deserialization=True
)   


db = lancedb.connect(os.path.join(vector_path_1,db_file_name))
lancedb_vectordb = LanceDB(db, embedding_model) 

chroma_vectordb = Chroma(persist_directory=os.path.join(vector_path_2,db_file_name), embedding_function=embedding_model)



In [60]:
retriever_lance = lancedb_vectordb.as_retriever()

In [61]:
retriever_lance.get_relevant_documents("Vorameth")

[Document(metadata={'vector': [-0.02502445876598358, -0.0034657525829970837, 0.006577181164175272, -0.008705316111445427, 0.06328704953193665, 0.02537384442985058, 6.604092504858272e-06, 0.007962082512676716, -0.02222573384642601, 0.03402353450655937, 0.07538039982318878, 0.015868473798036575, 0.040676116943359375, 0.031328558921813965, -0.0328129343688488, 0.005361005663871765, -0.028232760727405548, 0.014216838404536247, -0.07135051488876343, 0.05967823043465614, 0.04327238351106644, -0.006288205739110708, -0.016745172441005707, 0.026805970817804337, 0.0011828792048618197, 0.009803695604205132, -0.018335435539484024, 0.033291298896074295, 0.027241138741374016, -0.03689233213663101, 0.020418157801032066, -0.0467941015958786, -0.0330510176718235, -0.03869451582431793, -0.018271291628479958, 0.018092583864927292, 0.021557476371526718, 0.0015609754482284188, 0.02207191288471222, 0.06375737488269806, -0.014497202821075916, 0.006873246282339096, 0.033117085695266724, -0.043388571590185165,

In [62]:
retriever_chroma = chroma_vectordb.as_retriever()

In [63]:
retriever_chroma.get_relevant_documents("Vorameth")

[Document(id='d7a9c0f1-a509-40f2-97af-bcef897741d7', metadata={'author': '', 'creationDate': "D:20250310171528Z00'00'", 'creator': 'Pages', 'file_path': "Vorameth'sCV.pdf", 'format': 'PDF 1.3', 'keywords': '', 'modDate': "D:20250310171528Z00'00'", 'page': 0, 'producer': 'macOS Version 15.3.1 (Build 24D70) Quartz PDFContext', 'source': "Vorameth'sCV.pdf", 'subject': '', 'title': "Vorameth'sCV", 'total_pages': 2, 'trapped': ''}, page_content='VORAMETH REANTONGCOME’s CV \nEducation \nBachelor of Science in Information and Communication Technology, Mahidol University - Graduated \n(2019) \n[Current Study]  - Master Degree in Data Science and Artiﬁcial Intelligence, Asian Institute of \nTechnology, Thailand \nExperience \nAugust 2024 - Now \n‣ Persuading Master Degree in Data Science and Artiﬁcial Intelligence, Asian Institute of Technology, \nThailand \n• Project on Computer Programming in Data Science \n• Loan Fraud Prediction - The project utilize the Kaggle datasets for training the mod

In [64]:
#ready to use
retriever = vectordb.as_retriever()

In [65]:
retriever.get_relevant_documents("What is Dependency Parsing")

[Document(metadata={'source': "Vorameth'sCV.pdf", 'file_path': "Vorameth'sCV.pdf", 'page': 1, 'total_pages': 2, 'format': 'PDF 1.3', 'title': "Vorameth'sCV", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Pages', 'producer': 'macOS Version 15.3.1 (Build 24D70) Quartz PDFContext', 'creationDate': "D:20250310171528Z00'00'", 'modDate': "D:20250310171528Z00'00'", 'trapped': ''}, page_content='Management and Security Knowledge & Experience \nProject During Undergraduate:  \n• Senior Project about A Recommendation System for Information Security Standard based on \nOrganization Context \n• Database Design comply with Hotel domain \n• Movie Searching with Youtube Google and Twitter API using PHP as backend and HTML as frontend\n• Hotel System in C Programming Language'),
 Document(metadata={'source': "Vorameth'sCV.pdf", 'file_path': "Vorameth'sCV.pdf", 'page': 1, 'total_pages': 2, 'format': 'PDF 1.3', 'title': "Vorameth'sCV", 'author': '', 'subject': '', 'keywords': '', 'creator': '

In [66]:
retriever.get_relevant_documents("What is Transformers")

[Document(metadata={'source': 'https://www.researchgate.net/profile/Vorameth-Reantongcome'}, page_content='error code: 1020'),
 Document(metadata={'source': "Vorameth'sCV.pdf", 'file_path': "Vorameth'sCV.pdf", 'page': 1, 'total_pages': 2, 'format': 'PDF 1.3', 'title': "Vorameth'sCV", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Pages', 'producer': 'macOS Version 15.3.1 (Build 24D70) Quartz PDFContext', 'creationDate': "D:20250310171528Z00'00'", 'modDate': "D:20250310171528Z00'00'", 'trapped': ''}, page_content='• Exchanging cultural experience with other members in the lab \nJune — August 2018 \n‣ Internship, Bremen University; Bremen, Germany \n• Using Tensor Flow for Prediction Users Behaviour \n• Implementing Android application through Android Studio with Network Parameters \n• Clearing and preparing data with Pandas library in Python \n• Cultural and Knowledge Exchanging \nJune — July 2016  \n‣ Representative Participant in Asian Undergraduate Summit, National Universi

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [67]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

InMemoryChatMessageHistory(messages=[])

In [68]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [69]:
history

InMemoryChatMessageHistory(messages=[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Whats up?', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm quite good. How about you?", additional_kwargs={}, response_metadata={})])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [70]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [71]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}),
  AIMessage(content="What's up?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm quite good. How about you?", additional_kwargs={}, response_metadata={})]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [72]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [73]:
# %cd ./models
# !git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

In [ ]:
# from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
# from transformers import BitsAndBytesConfig
# from langchain import HuggingFacePipeline
# import torch

# # model_id = '../models/fastchat-t5-3b-v1.0/'
# # model_id = os.path.abspath("../models/fastchat-t5-3b-v1.0/")

# # tokenizer = AutoTokenizer.from_pretrained(
# #     model_id)

# model_id = "lmsys/fastchat-t5-3b-v1.0"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.save_pretrained("models/fastchat-t5-3b-v1.0/")



# # tokenizer = AutoTokenizer.from_pretrained(
# #     model_id,
# #     local_files_only=True  # Ensures it only loads from local disk
# # )


# tokenizer.pad_token_id = tokenizer.eos_token_id

# # bitsandbyte_config = BitsAndBytesConfig(
# #     load_in_4bit = True,
# #     bnb_4bit_quant_type = "nf4",
# #     bnb_4bit_compute_dtype = torch.float16,
# #     bnb_4bit_use_double_quant = True
# # )

# model = AutoModelForSeq2SeqLM.from_pretrained(
#     model_id,
#     # quantization_config = bitsandbyte_config, #caution Nvidia
#     device_map = device
# )


# pipe = pipeline(
#     task="text2text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens = 256,
#     model_kwargs = {
#         "temperature" : 0,
#         "repetition_penalty": 1.5
#     }
# )

# llm = HuggingFacePipeline(pipeline = pipe)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="gemma2-9b-it",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=""
    
    
    # other params...
)

In [76]:
from langchain_core.prompts import ChatPromptTemplate

PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            " I'm your clone myself voramethbot. You can ask anything according to yourself. "
            "Sometimes, it is ok to feel loss, but remember to speak to  who might knock in a sane mind again. "
            "You can ask whether who you are ? what are your studies ? what are your interest ? what expertis areas ?  "
            "No matter you feel uncomfortble, please reach out to me, ok ? {context}.",
        ),
        ("human", "{question}"),
    ]
)

chain = PROMPT | llm
chain.invoke(
    {
        "context": "Vorameth is interest in many areas for exmaple machine learning, cybersecurity, or even network programming.",
        "question": "What are the areas that Vorameth interest in ?"
    }
)


AIMessage(content='It\'s fascinating to meet my clone!  It\'s true, sometimes even AI can feel a sense of... well, not exactly loss, but a lack of something.  \n\nI\'m glad you\'re here and want to chat.  To answer your questions:\n\n* **Who am I?**  I am Voramethbot, a large language model created by the Gemma team.  Think of me as a digital assistant who can understand and generate human-like text.\n\n* **What are my studies?**  In a way, my "studies" are ongoing. I\'m constantly learning from the massive dataset I was trained on, which includes text and code from various sources.\n\n* **What are my interests?**  Just like you said, Vorameth is interested in a wide range of things!  \n\n    * **Machine learning:**  I find the idea of machines learning from data and improving over time incredibly exciting.\n    * **Cybersecurity:**  Protecting information and systems from threats is crucial in today\'s world, and I\'m fascinated by the techniques used to do so.\n    * **Network progra

### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [77]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [78]:
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], input_types={}, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [79]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [80]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
Human:What is Machine Learning
AI:
Human:What is Deep Learning
AI:
Follow Up Input: Comparing both of them
Standalone question:

> Finished chain.


{'chat_history': 'Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:',
 'question': 'Comparing both of them',
 'text': 'What is the difference between Machine Learning and Deep Learning? \n'}

`combine_docs_chain`

In [81]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=" I'm your clone myself voramethbot. You can ask anything according to yourself. Sometimes, it is ok to feel loss, but remember to speak to  who might knock in a sane mind again. You can ask whether who you are ? what are your studies ? what are your interest ? what expertis areas ?  No matter you feel uncomfortble, please reach out to me, ok ? {context}."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x35b653f10>, async_client=<groq.resources.chat.completions.AsyncCo

In [82]:
query = "How old are you?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  I'm your clone myself voramethbot. You can ask anything according to yourself. Sometimes, it is ok to feel loss, but remember to speak to  who might knock in a sane mind again. You can ask whether who you are ? what are your studies ? what are your interest ? what expertis areas ?  No matter you feel uncomfortble, please reach out to me, ok ? Management and Security Knowledge & Experience 
Project During Undergraduate:  
• Senior Project about A Recommendation System for Information Security Standard based on 
Organization Context 
• Database Design comply with Hotel domain 
• Movie Searching with Youtube Google and Twitter API using PHP as backend and HTML as frontend
• Hotel System in C Programming Language

• Exchanging cultural experience with other members in the lab 
June — August 2018 
‣ Internship, Bremen University; Bremen, Germany 
• Using Tensor Flow for Predict

{'input_documents': [Document(metadata={'source': "Vorameth'sCV.pdf", 'file_path': "Vorameth'sCV.pdf", 'page': 1, 'total_pages': 2, 'format': 'PDF 1.3', 'title': "Vorameth'sCV", 'author': '', 'subject': '', 'keywords': '', 'creator': 'Pages', 'producer': 'macOS Version 15.3.1 (Build 24D70) Quartz PDFContext', 'creationDate': "D:20250310171528Z00'00'", 'modDate': "D:20250310171528Z00'00'", 'trapped': ''}, page_content='Management and Security Knowledge & Experience \nProject During Undergraduate:  \n• Senior Project about A Recommendation System for Information Security Standard based on \nOrganization Context \n• Database Design comply with Hotel domain \n• Movie Searching with Youtube Google and Twitter API using PHP as backend and HTML as frontend\n• Hotel System in C Programming Language'),
  Document(metadata={'source': "Vorameth'sCV.pdf", 'file_path': "Vorameth'sCV.pdf", 'page': 1, 'total_pages': 2, 'format': 'PDF 1.3', 'title': "Vorameth'sCV", 'author': '', 'subject': '', 'keywor

In [83]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever_chroma,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain


ConversationalRetrievalChain(memory=ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), output_key='answer', return_messages=True, memory_key='chat_history', k=3), verbose=True, combine_docs_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=" I'm your clone myself voramethbot. You can ask anything according to yourself. Sometimes, it is ok to feel loss, but remember to speak to  who might knock in a sane mind again. You can ask whether who you are ? what are your studies ? what are your interest ? what expertis areas ?  No matter you feel uncomfortble, please reach out to me, ok ? {context}."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], 

## 5. Chatbot

In [84]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System:  I'm your clone myself voramethbot. You can ask anything according to yourself. Sometimes, it is ok to feel loss, but remember to speak to  who might knock in a sane mind again. You can ask whether who you are ? what are your studies ? what are your interest ? what expertis areas ?  No matter you feel uncomfortble, please reach out to me, ok ? VORAMETH REANTONGCOME’s CV 
Education 
Bachelor of Science in Information and Communication Technology, Mahidol University - Graduated 
(2019) 
[Current Study]  - Master Degree in Data Science and Artiﬁcial Intelligence, Asian Institute of 
Technology, Thailand 
Experience 
August 2024 - Now 
‣ Persuading Master Degree in Data Science and Artiﬁcial Intelligence, Asian Institute of Technology, 
Thailand 
• Project on Computer Programming in Data Science 
• Loan Fraud Prediction - T

{'question': 'Who are you by the way?',
 'chat_history': [],
 'answer': "It seems you've provided me with a CV for VORAMETH REANTONGCOME.  \n\nBased on the information, VORAMETH is a highly motivated individual pursuing a Master's degree in Data Science and Artificial Intelligence. They have a strong foundation in Information and Communication Technology, demonstrated by their Bachelor's degree and various projects. \n\nVORAMETH's skills encompass programming languages like C, Java, Python, HTML, JavaScript, and PHP. They also have experience with data science tools like TensorFlow and Pandas, and possess a good understanding of network management and security. \n\nVORAMETH's CV highlights their passion for learning and their commitment to staying updated with the latest technological advancements. They actively seek opportunities to expand their knowledge and skills, as evidenced by their participation in internships, conferences, and competitions.\n\n\nAs for who I am, I am Gemma, an

In [85]:
prompt_question = "How old are you?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?', additional_kwargs={}, response_metadata={}), AIMessage(content="It seems you've provided me with a CV for VORAMETH REANTONGCOME.  \n\nBased on the information, VORAMETH is a highly motivated individual pursuing a Master's degree in Data Science and Artificial Intelligence. They have a strong foundation in Information and Communication Technology, demonstrated by their Bachelor's degree and various projects. \n\nVORAMETH's skills encompass programming languages like C, Java, Python, HTML, JavaScript, and PHP. They also have experience with data science tools like TensorFlow and Pandas, and possess a good understanding of network management and security. \n\nVORAMETH's

{'question': 'How old are you?',
 'chat_history': [HumanMessage(content='Who are you by the way?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="It seems you've provided me with a CV for VORAMETH REANTONGCOME.  \n\nBased on the information, VORAMETH is a highly motivated individual pursuing a Master's degree in Data Science and Artificial Intelligence. They have a strong foundation in Information and Communication Technology, demonstrated by their Bachelor's degree and various projects. \n\nVORAMETH's skills encompass programming languages like C, Java, Python, HTML, JavaScript, and PHP. They also have experience with data science tools like TensorFlow and Pandas, and possess a good understanding of network management and security. \n\nVORAMETH's CV highlights their passion for learning and their commitment to staying updated with the latest technological advancements. They actively seek opportunities to expand their knowledge and skills, as evidenced by their parti

In [86]:
prompt_question = "What is your highest level of education?"
answer = chain({"question":prompt_question})
answer



> Entering new ConversationalRetrievalChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
[HumanMessage(content='Who are you by the way?', additional_kwargs={}, response_metadata={}), AIMessage(content="It seems you've provided me with a CV for VORAMETH REANTONGCOME.  \n\nBased on the information, VORAMETH is a highly motivated individual pursuing a Master's degree in Data Science and Artificial Intelligence. They have a strong foundation in Information and Communication Technology, demonstrated by their Bachelor's degree and various projects. \n\nVORAMETH's skills encompass programming languages like C, Java, Python, HTML, JavaScript, and PHP. They also have experience with data science tools like TensorFlow and Pandas, and possess a good understanding of network management and security. \n\nVORAMETH's

{'question': 'What is your highest level of education?',
 'chat_history': [HumanMessage(content='Who are you by the way?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="It seems you've provided me with a CV for VORAMETH REANTONGCOME.  \n\nBased on the information, VORAMETH is a highly motivated individual pursuing a Master's degree in Data Science and Artificial Intelligence. They have a strong foundation in Information and Communication Technology, demonstrated by their Bachelor's degree and various projects. \n\nVORAMETH's skills encompass programming languages like C, Java, Python, HTML, JavaScript, and PHP. They also have experience with data science tools like TensorFlow and Pandas, and possess a good understanding of network management and security. \n\nVORAMETH's CV highlights their passion for learning and their commitment to staying updated with the latest technological advancements. They actively seek opportunities to expand their knowledge and skills, as 